In [ ]:
load_ext autoreload

In [ ]:
autoreload 2

In [ ]:
import makegridslibrary as me
from pypci import pci

import pyphi

import scipy.io as scio
import numpy as np

import itertools
import pickle
import random

import matplotlib.pyplot as plt
from matplotlib import cm
import matplotlib
import solarized
from IPython.core.display import display, HTML

# matplotlib.rcParams.update(matplotlib.rcParamsDefault)
# solarized.light()
# solarized.dark()


In [ ]:
##########################################################################################################################
## First load \Phi results from Pyphi
##########################################################################################################################

# Import probability of occuppying each state and use it as weight for:

# * \Phi
# * \sum \phi
# * \sum \Phi * \phi

def read_prob_state(file_name):
    ps = np.transpose(scio.loadmat(file_name)['all_states'], (0, 2, 3, 1))
    nps = ps.shape
    N = 0 # only the 5 nodes
    pss = np.zeros((nps[1], 32))
    for itemp in range(nps[1]):
        pss[itemp, :] = np.mean(
            np.array([ np.histogram(ps[N, itemp, run, :], range(0,33), density=True)[0] for run in range(nps[2]) ]),
            axis = 0
        )
    return pss

# grid_states = '/data/nsdm/pyphi/dynamics/all_states_5_nodes_non_cm_grid_fpp_T_0.0_1.4_15.mat'
# rand_states = '/data/nsdm/pyphi/dynamics/all_states_5_nodes_non_cm_rand_fpp_T_0.0_1.4_15.mat'

grid_states = '/data/nsdm/pyphi/dynamics/all_states_5_nodes_non_cm_grid_fpp_T_0.0_2.8_12.mat'
rand_states = '/data/nsdm/pyphi/dynamics/all_states_5_nodes_non_cm_rand_fpp_T_0.0_2.8_12.mat'

# grid_states = '/data/nsdm/pyphi/dynamics/all_states_5_nodes_non_cm_grid_fpp_T_0.5_1.0_2.mat'
# rand_states = '/data/nsdm/pyphi/dynamics/all_states_5_nodes_non_cm_rand_fpp_T_0.5_1.0_2.mat'

grid_prob_state = read_prob_state(grid_states)
rand_prob_state = read_prob_state(rand_states)

# Load results from matching_LB.py

results_filename = '/data/nsdm/pyphi/fivenodes_noinput_gridVSrandom_ENTROPY_WEDGE_DIFFSUM_T_0_3_D_2_MAIN_SAVED_new.pickle'
#results_filename = '/data/nsdm/pyphi/fivenodes_noinput_gridVSrandom_ENTROPY_WEDGE_DIFFSUM_T_0_3_D_2_MAIN_SAVED.pickle'
trange = [0, 2.75]

# results_filename = '/data/nsdm/pyphi/fivenodes_barVSshuffled_gridVSrandom_ENTROPY_WEDGE_DIFFSUM_T_0_1_D_2_MAIN_SAVED.pickle'
# trange = [0, 1.4]

# results_filename = '/data/nsdm/pyphi/fivenodes_noinput_gridVSrandom_ENTROPY_WEDGE_DIFFSUM_T_0.5_1.0_D_2_L2_MAIN_SAVED.pickle'
# trange = [.5, 1.]

with open(results_filename, 'rb') as f: 
    results = pickle.load(f)
results = results[0]

ntemp = len(results['Grid'])
temperatures = np.linspace(trange[0], trange[1], ntemp)

do_plot = False
# do_plot = True

# ignore_homo = True
ignore_homo = False

# weight_state = True
weight_state = False

def operation(concepts, big_phi, operation_type = 'concepts * big_phi'):
    values = eval(operation_type)
    return (np.sum(values), np.std(values))

# network_labels = ['Random']
# network_labels = ['Grid']
network_labels = ['Grid', 'Random']
operation_types = ['concepts * big_phi', 'big_phi', 'concepts']

all_results =  {x: {y: np.zeros(ntemp) for y in operation_types} for x in network_labels}
all_results_std = {x: {y: np.zeros(ntemp) for y in operation_types} for x in network_labels}

for network_label in network_labels:
    
    if do_plot:
#         fig, axis = plt.subplots(int(len(temperatures)/2)+1, 2)
        fig, axis = plt.subplots(len(temperatures), 2)
        fig.set_size_inches(12,12)
        
    for operation_type in operation_types:
        
        for (itemp, temp) in enumerate(sorted(results[network_label])):
            
            print('%s, temp = %2.2f, operation: %s' % (network_label, temp, operation_type))
            
            this_result = results[network_label][temp]
            number_result = np.zeros(len(this_result))
            number_result_dict = {}
            
            phi = np.zeros(len(this_result))
            phi_std = np.zeros(len(this_result))
            for (istate, this_state) in enumerate(this_result):

                state_ok = len(this_result[this_state][1]) > 0

                if ignore_homo:
                    state_ok = state_ok & (sum(this_state[:5]) > 0 and sum(this_state[:5]) < len(this_state[:5]))

                if state_ok:
                    number_result[istate] = len(this_result[this_state][1])
                    number_result_dict[this_state] = len(this_result[this_state][1])

                    (stat_1, stat_2) = operation(np.array(this_result[this_state][1]), this_result[this_state][0], operation_type)
                    (phi[istate], phi_std[istate]) = (stat_1, stat_2)


            if do_plot: # and itemp%2 == 0:
    #             idx = int(itemp/2)
                idx = itemp
                axis[idx, 0].hist(phi)
                axis[idx, 0].axvline(x=np.mean(phi), color='r')
                axis[idx, 0].set_title('mean: %2.5f' % np.mean(phi))
                axis[idx, 0].set_ylabel('T: %2.2f' % temp)

            if weight_state:
                if network_label == 'Random':
                    phi = phi * rand_prob_state[itemp,:]
                    phi_std = phi_std * rand_prob_state[itemp,:]
                else:
                    phi = phi * grid_prob_state[itemp,:]
                    phi_std = phi_std * grid_prob_state[itemp,:]
                
            all_results[network_label][operation_type][itemp] = np.mean(phi)
            all_results_std[network_label][operation_type][itemp] = np.sqrt(np.mean(phi_std**2))

    if do_plot:
        plt.tight_layout()
#         fig.suptitle(network_label + ' : ' + operation_type)
        fig.suptitle(network_label)
        plt.subplots_adjust(top=0.9)
    
if do_plot:
    plt.show()
    
# print(all_matching)

In [ ]:
with open(results_filename, 'rb') as f: 
    results = pickle.load(f)
    


In [ ]:
len(results)


In [ ]:
import pylab as plt
import solarized

# t0 = 2
t0 = 0

markers = {'Grid': 'o', 'Random': 'P'}
b = solarized.COLOR["blue"]
r = solarized.COLOR["red"]
m = solarized.COLOR["magenta"]

network = 'Grid'
plt.figure(figsize = (17,6))
for (iop, operation) in enumerate(sorted(all_results[network])):

    plt.subplot(1,3,iop+1)
    plt.errorbar(temperatures[t0:], all_results[network][operation][t0:], yerr=all_results_std[network][operation][t0:], fmt=markers[network], color=b)
    plt.title(network + ' : ' + operation)
    plt.ylabel('Phi')
    plt.xlabel('Noise')
plt.show()    


network = 'Random'
plt.figure(figsize = (17,6))
for (iop, operation) in enumerate(sorted(all_results[network])):

    plt.subplot(1,3,iop+1)
    plt.errorbar(temperatures[t0:], all_results[network][operation][t0:], yerr=all_results_std[network][operation][t0:], fmt=markers[network], color=b)
    plt.title(network + ' : ' + operation)
    plt.ylabel('Phi')
    plt.xlabel('Noise')
plt.show()    


plt.figure(figsize = (17,6))

for (iop, operation) in enumerate(sorted(all_results[network])):

    plt.subplot(1,3,iop+1)
    y = all_results['Grid'][operation][t0:] - all_results['Random'][operation][t0:]
    yerr = np.sqrt(np.sum(all_results_std['Grid'][operation][t0:]**2 + all_results_std['Random'][operation][t0:]**2))
    plt.errorbar(temperatures[t0:], y, yerr=yerr, fmt='s', color=m)
    plt.title(network + ' : ' + operation)
    plt.ylabel('Phi')
    plt.xlabel('Noise')
        
plt.show()    



In [ ]:
##########################################################################################################################
## Specify mutual information results that should be used
##########################################################################################################################

from matplotlib.pyplot import savefig
from subprocess import call

fig_out_dir = '/home/leonardo/projects/nsdm-pyphi/docs/auto'

mi_input_dir = '/data/nsdm/pyphi/dynamics/mi'

mi_input_files = ['measure_d1_v2_grid_rand_non_test_T_0.0_2.8.pickle']
# mi_input_files = ['measure_dte_v2_grid_rand_non_test_T_0.0_2.8.pickle']

# mi_input_files = ['measure_dte_v2_grid_rand_non_test_T_0.5_1.0.pickle']

b = solarized.COLOR["blue"]
r = solarized.COLOR["red"]
m = solarized.COLOR["magenta"]

markers = {'Grid': 'o', 'Random': 'x'}
fcolor = {'Grid': m, 'Random': 'none'}

t0 = 2
# t0 = 0

imn = 1

ts = temperatures


In [ ]:
##########################################################################################################################
## Show MI information per temperature
##########################################################################################################################

do_simple_mi_plot = False
# do_simple_mi_plot = True

do_plot_mi_per_temp = False
# do_plot_mi_per_temp = True

do_plot_diff_grid_rand = False
# do_plot_diff_grid_rand = True


def plot_mi_per_temp(net):
    
    imn = 1
    
    nt = len(net[2])
    nruns = len(net[2][0])
    n_mecs = net[2][0][0][imn].shape[0]
    # plt.plot([(n+1) * net[2][t][r][2][0] - net[2][t][r][2][n] for n in range(n_mecs)])
    t_hs = []
    tcolors = cm.Paired([x for x in np.linspace(0., 1., nt)])
    for (t, color) in zip(range(nt), tcolors):
        runs = np.mean(np.array([net[2][t][r][imn] for r in range(nruns)]), axis=0)
        runs_std = np.std(np.array([net[2][t][r][imn] for r in range(nruns)]), axis=0)
        t_h = plt.errorbar(range(1,n_mecs+1), runs, yerr=runs_std, color=color)
        ext_m0 = range(1, n_mecs+1)
        plt.plot(ext_m0, [ext * runs[0] for ext in ext_m0], color=color, ls='--')
        t_hs.append(t_h)
    plt.legend(t_hs, [str(t) for t in range(nt)])



all_phi = {'non': (all_results, all_results_std)}
    
if do_plot_diff_grid_rand:    
    
    fig_phi_root = 'diff_rand_grid'
    fig_files_phi = []
    
    for (iop, operation) in enumerate(['big_phi', 'concepts']):    
        
        # Difference in big phi and sunm opf concepts
        plt.figure(figsize = (17,6))

        plt.subplot(1,3,1)
        netgrid = all_phi['non'][0]['Grid'][operation]
        netrand = all_phi['non'][0]['Random'][operation]
        plt.errorbar(ts[t0:], netgrid[t0:]-netrand[t0:], 
                     fmt='s', color=m, markersize=10)
        plt.ylabel('Grid - Rand')
        plt.xlabel('Temperature')
        plt.title('Measure: %s, Tau: %d' % (operation, tau))
        
        fig_name = '%s/%s_%s.png' % (fig_out_dir, fig_phi_root, operation)
        fig_files_phi.append(fig_name)
        savefig(fig_name)

        plt.show()
        
    if len(fig_files_corr):
        call(["convert"] + fig_files_phi + ['-append', '%s/%s.png' % (fig_out_dir, fig_phi_root)])

for mi_input_file in mi_input_files:

    display(HTML("<center><h1 style='font-size: %dpx;height: 100;'>%s</h1></center>" % (20, mi_input_file)))
        
    with open('%s/%s' % (mi_input_dir, mi_input_file), 'rb') as f:
        mis = pickle.load(f)

    fig_files = []
    fig_out_root = 'corr_phi_mi_no_input'
    
    fig_files_corr = []
    fig_out_corr_root = 'corr_phi_mi_no_input_diff_grid_rand'
    
    for tau in mis:

        networks_d = {'Grid': {'non': mis[tau]['non_grid']}, 'Random': {'non': mis[tau]['non_rand']}}

#         networks_d = {'Grid': {'non': mis[tau]['non_grid']}}
#         networks_d = {'Random': {'non': mis[tau]['non_rand']}}
        
        
        # mutual information for each network
        if do_plot_mi_per_temp:
            for (inet, network_label) in enumerate(sorted(networks_d)):

                plt.figure(figsize = (17,6))

                network = networks_d[network_label]

                for (ilabel, label) in enumerate(['non']):
                    ax = plt.subplot(1,2,ilabel+1)
                    plot_mi_per_temp(network[label])
                    ax.set_title('net: %s, input: %s, tau: %d' % (network_label, label, tau))
                plt.show()

                
        # Degeneracy-like measure for each network
        if do_simple_mi_plot:
            plt.figure(figsize = (17,6))
            for (inet, network_label) in enumerate(sorted(networks_d)):

                network = networks_d[network_label]

                plt.subplot(1,2,inet+1)
                plt.errorbar(ts[t0:], network['non'][0][t0:], yerr=network['non'][1][t0:], 
                             fmt=markers[network_label], color=b, markersize=10)
                plt.title(network_label + ' tau = ' + str(tau))
                plt.ylabel(measure)
                plt.xlabel('Temperature')

            plt.show()    
        

        if do_plot_diff_grid_rand:
            # Difference in degeneracy-like measure
            plt.figure(figsize = (17,6))

            plt.subplot(1,3,1)
            netgrid = networks_d['Grid']
            netrand = networks_d['Random']
            plt.errorbar(ts[t0:], netgrid['non'][0][t0:]-netrand['non'][0][t0:], 
                         fmt='s', color=m, markersize=10)
            plt.ylabel('Grid - Rand')
            plt.xlabel('Temperature')
            plt.title('Measure: %s, Tau: %d' % (measure, tau))

            for (iop, operation) in enumerate(['big_phi', 'concepts']):    

                netgrid_phi = all_phi['non'][0]['Grid'][operation]
                netrand_phi = all_phi['non'][0]['Random'][operation]
                
                ax = plt.subplot(1,3,iop+2)

#                 scatter_kwargs = {"zorder":100}
#                 error_kwargs = {"lw":.5, "zorder":0}

                X = netgrid_phi[t0:]-netrand_phi[t0:]
#                 Y = np.log(netgrid['non'][0][t0:])-np.log(netrand['non'][0][t0:])
                Y = netgrid['non'][0][t0:] - netrand['non'][0][t0:]
                C = cm.jet(np.linspace(0, 1, len(temperatures[t0:])))

                im = plt.scatter(X,Y,c=C, marker='s',s=80, **scatter_kwargs)
#                 plt.errorbar(X, Y, fmt=markers[network], **error_kwargs)

                plt.ylabel('log(%s)' % measure)
                plt.xlabel(operation)
                plt.title('Measure: %s, Tau: %d' % (operation, tau))

                ax.autoscale(enable=True, axis='x', tight=True)
#                 xlims = [a*b for (a,b) in zip([.95, 1.05], [np.min(X), np.max(X)])]
                ax.set_xlim([-.001, .001]) 

#             fig_name = '%s/%s_%s_%d.png' % (fig_out_dir, fig_out_corr_root, measure, tau)
#             fig_files_corr.append(fig_name)
#             savefig(fig_name)
    
            plt.show()            
            

#     if len(fig_files):
#         call(["convert"] + fig_files + ['-append', '%s/%s.png' % (fig_out_dir, fig_out_root)])

#     if len(fig_files_corr):
#         call(["convert"] + fig_files_corr + ['-append', '%s/%s.png' % (fig_out_dir, fig_out_corr_root)])        


In [ ]:
##########################################################################################################################
## Correlate MI results with Pyphi results
##########################################################################################################################

import scipy 

fig_out_root = 'corr_phi_mi_no_input'

t0 = 2
# t0 = 0

# mi_input_files = ['measure_d1_v2_grid_rand_non_test_T_0.0_2.8.pickle',
#                  'measure_dte_v2_grid_rand_non_test_T_0.0_2.8.pickle']

mi_input_files = ['measure_d1_mice_grid_rand_non_test_T_0.0_2.8_new2.pickle']

# mi_input_files = ['measure_d1_mice_grid_rand_non_test_T_0.0_2.8_new.pickle',
# 'measure_d1_cause_grid_rand_non_test_T_0.0_2.8_new.pickle',
# 'measure_d1_effect_grid_rand_non_test_T_0.0_2.8_new.pickle']

# mi_input_files = ['measure_d1_v2_grid_rand_non_test_T_0.0_2.8.pickle']
# mi_input_files = ['measure_dte_v2_grid_rand_non_test_T_0.0_2.8.pickle']

# mi_input_files = ['measure_d1_v2_grid_rand_non_test_T_0.0_2.8.pickle',
#                  'measure_dte_v2_grid_rand_non_test_T_0.0_2.8.pickle']

# mi_input_files = ['measure_dte_v2_grid_rand_non_test_T_0.5_1.0.pickle']

markers = {'Grid': 'o-', 'Random': 'x--'}
fcolor = {'Grid': m, 'Random': 'none'}

use_average_mi = True
# use_average_mi = False

input_types = ['non']

def sum_mi_over_mechanisms(all_measures):
    
    all_mi = all_measures[2]

    ntemp = len(all_mi)
    nruns = len(all_mi[0])
    maxmec = len(all_mi[0][0][imn])+1

#     Y_mean = [np.mean([np.sum(all_mi[t][r][imn]) for r in range(nruns)]) for t in range(ntemp)]    
#     Y_std = [np.std([np.sum(all_mi[t][r][imn]) for r in range(nruns)]) for t in range(ntemp)]  

    Y_mean = [np.mean([np.sum(m/n for m,n in zip(all_mi[t][r][imn], range(1,maxmec))) for r in range(nruns)]) for t in range(ntemp)]    
    Y_std = [np.std([np.sum(m/n for m,n in zip(all_mi[t][r][imn], range(1,maxmec))) for r in range(nruns)]) for t in range(ntemp)]    

    return Y_mean, Y_std

def log_sum_mi_over_mechanisms(all_measures):
    Y_mean, Y_std = sum_mi_over_mechanisms(all_measures)
    return np.log(Y_mean), Y_std

def log_degeneracy(all_measures):
    return np.log(np.abs(all_measures[0])), all_measures[1]

def degeneracy(all_measures):
    return all_measures[0], all_measures[1]

# all_measures = [sum_mi_over_mechanisms, degeneracy]
all_measures = [sum_mi_over_mechanisms, log_sum_mi_over_mechanisms, degeneracy, log_degeneracy]
# all_measures = [sum_mi_over_mechanisms]

all_phi = {'non': (all_results, all_results_std)}
    
for mi_input_file in mi_input_files:

    measure_class = '_'.join(mi_input_file.split('_')[:3])

    display(HTML("<center><h1 style='font-size: %dpx;height: 100;'>%s</h1></center>" % (20, measure_class)))

    with open('%s/%s' % (mi_input_dir, mi_input_file), 'rb') as f:
        mis = pickle.load(f)

    all_mi = {tau: {'Grid': {input_type: mis[tau][input_type + '_grid'] for input_type in input_types}, 
                    'Random': {input_type: mis[tau][input_type + '_rand'] for input_type in input_types}}
                  for tau in mis}
        
    fig_files = []

    for measure in all_measures:

        # correlate \Phi, \sum \phi and their product with MI-measures with 
        for input_type in input_types:

            phi_for_input = all_phi[input_type]
            
            fig, axs = plt.subplots(1, 2, sharey=True, figsize = (17,6))
            axs[0].set_ylabel(measure.__name__)
            axs_xlims = [[0, 0] for ax in axs]

            for (inet, network) in enumerate(sorted(phi_for_input[0])):

#                 cmap = matplotlib.cm.get_cmap('default', len(mis)).colors
                for ax in axs: ax.set_prop_cycle(None)
                
#                 for (iop, operation) in enumerate(sorted(phi_for_input[0][network])):
                for (iop, operation) in enumerate(['big_phi', 'concepts']):

                    ax = axs[iop]
            
                    X = phi_for_input[0][network][operation][t0:]
                    X_err = phi_for_input[1][network][operation][t0:]

                    if np.min(X) < axs_xlims[iop][0]: axs_xlims[iop][0] = np.min(X)
                    if np.max(X) > axs_xlims[iop][1]: axs_xlims[iop][1] = np.max(X)
                    
                    for (itau, tau) in enumerate(mis):

                        scatter_kwargs = {"zorder":100}
#                         error_kwargs = {"lw":.5, "zorder":0, "color":cmap[itau, :]}
                        error_kwargs = {"lw":.5, "zorder":0}

                        Y, Y_err = measure(all_mi[tau][network][input_type])
                        Y, Y_err = Y[t0:], Y_err[t0:]

                        C = cm.jet(np.linspace(0, 1, len(temperatures[t0:])))

                        t_corr, p_corr = scipy.stats.pearsonr(X, Y)
#                         t_corr, p_corr = scipy.stats.spearmanr(X, Y)
                        label = "%s: tau %d (%2.2f)" % (network, tau, t_corr)

                        im = ax.scatter(X, Y, c=C, marker=markers[network][0], **scatter_kwargs)
                        ax.errorbar(X, Y, yerr=Y_err, xerr=X_err, fmt=markers[network], label=label, **error_kwargs)

                    ax.set_xlabel(operation)

                    handles, labels = ax.get_legend_handles_labels()
                    ax.legend(handles, labels)

                    ax.set_title("%s (%s)" % (input_type, operation))
            
                xlims = [a*b for (a,b) in zip([.95, 1.05], [np.min(X), np.max(X)])]
                ax.set_xlim(xlims) 

            m = cm.ScalarMappable(cmap=cm.jet)
            m.set_array(2*temperatures[t0:])
            fig.colorbar(m,  ax=axs.ravel().tolist(), orientation='horizontal', label='temperature',
                        fraction=0.046) # , pad=0.04

#             handles, labels = axs[0].get_legend_handles_labels()
#             plt.figlegend( handles, labels, loc = 'upper center', ncol=5, labelspacing=0. )

            plt.show() 

#                 fig_name = '%s/%s_%s_%s.png' % (fig_out_dir, fig_out_root, measure, input_type, network)
#                 fig_files.append(fig_name)
#                 savefig(fig_name)



#     if len(fig_files):
#         call(["convert"] + fig_files + ['-append', '%s/%s.png' % (fig_out_dir, fig_out_root)])



In [ ]:
##########################################################################################################################
## Correlate difference between grid and random in the MI results vs differences in Pyphi results
##########################################################################################################################

fig_out_root = 'corr_phi_mi_diff_no_input'

def sum_mi_over_mechanisms(all_measures):
    
    all_mi = all_measures[2]

    ntemp = len(all_mi)
    nruns = len(all_mi[0])
    maxmec = len(all_mi[0][0][imn])+1

#     Y_mean = [np.mean([np.sum(all_mi[t][r][imn]) for r in range(nruns)]) for t in range(ntemp)]    
#     Y_std = [np.std([np.sum(all_mi[t][r][imn]) for r in range(nruns)]) for t in range(ntemp)]  

    Y_mean = [np.mean([np.sum(m/n for m,n in zip(all_mi[t][r][imn], range(1,maxmec))) for r in range(nruns)]) for t in range(ntemp)]    
    Y_std = [np.std([np.sum(m/n for m,n in zip(all_mi[t][r][imn], range(1,maxmec))) for r in range(nruns)]) for t in range(ntemp)]    

    return Y_mean, Y_std

def log_sum_mi_over_mechanisms(all_measures):
    Y_mean, Y_std = sum_mi_over_mechanisms(all_measures)
    return np.log(Y_mean), Y_std

def log_degeneracy(all_measures):
    return np.log(np.abs(all_measures[0])), all_measures[1]

def degeneracy(all_measures):
    return all_measures[0], all_measures[1]

# all_measures = [sum_mi_over_mechanisms, degeneracy]
all_measures = [sum_mi_over_mechanisms, log_sum_mi_over_mechanisms, degeneracy, log_degeneracy]
# all_measures = [sum_mi_over_mechanisms]

all_phi = {'non': (all_results, all_results_std)}
    
for mi_input_file in mi_input_files:

    measure_class = '_'.join(mi_input_file.split('_')[:3])

    display(HTML("<center><h1 style='font-size: %dpx;height: 100;'>%s</h1></center>" % (20, measure_class)))

    with open('%s/%s' % (mi_input_dir, mi_input_file), 'rb') as f:
        mis = pickle.load(f)

    all_mi = {tau: {'Grid': {input_type: mis[tau][input_type + '_grid'] for input_type in input_types}, 
                    'Random': {input_type: mis[tau][input_type + '_rand'] for input_type in input_types}}
                  for tau in mis}
        
    fig_files = []

    for measure in all_measures:

        # correlate \Phi, \sum \phi and their product with MI-measures with 
        for input_type in input_types:

            phi_for_input = all_phi[input_type]
            
            fig, axs = plt.subplots(1, 2, sharey=True, figsize = (17,6))
            axs[0].set_ylabel(measure.__name__)
            axs_xlims = [[0, 0] for ax in axs]

#                 cmap = matplotlib.cm.get_cmap('default', len(mis)).colors
            for ax in axs: ax.set_prop_cycle(None)

#                 for (iop, operation) in enumerate(sorted(phi_for_input[0][network])):
            for (iop, operation) in enumerate(['big_phi', 'concepts']):

                ax = axs[iop]

                X = phi_for_input[0]['Grid'][operation][t0:] - phi_for_input[0]['Random'][operation][t0:]
#                 X_err = np.sqrt(np.sum(phi_for_input[1]['Grid'][operation][t0:]**2 + phi_for_input[1]['Random'][operation][t0:]**2))
                X_err = 0

                if np.min(X) < axs_xlims[iop][0]: axs_xlims[iop][0] = np.min(X)
                if np.max(X) > axs_xlims[iop][1]: axs_xlims[iop][1] = np.max(X)

                for (itau, tau) in enumerate(mis):

                    scatter_kwargs = {"zorder":100}
#                         error_kwargs = {"lw":.5, "zorder":0, "color":cmap[itau, :]}
                    error_kwargs = {"lw":.5, "zorder":0}

                    Y_grid, Y_err_grid = measure(all_mi[tau]['Grid'][input_type])
                    Y_random, Y_err_random = measure(all_mi[tau]['Random'][input_type])

                    Y = [yg-yr for (yg,yr) in zip(Y_grid,Y_random)]
                    Y_err = [np.sqrt(np.sum(yg**2 + yr**2)) for (yg,yr) in zip(Y_err_grid,Y_err_random)]

                    Y, Y_err = Y[t0:], Y_err[t0:]
                    
#                     Y_err = 0

                    C = cm.jet(np.linspace(0, 1, len(temperatures[t0:])))

                    t_corr, p_corr = scipy.stats.pearsonr(X, Y)
#                         t_corr, p_corr = scipy.stats.spearmanr(X, Y)
                    label = "tau %d (%2.2f)" % (tau, t_corr)

                    im = ax.scatter(X, Y, c=C, marker='s', **scatter_kwargs)
                    ax.errorbar(X, Y, yerr=Y_err, xerr=X_err, fmt='s-', label=label, **error_kwargs)

                ax.set_xlabel(operation)

                handles, labels = ax.get_legend_handles_labels()
                ax.legend(handles, labels)

                ax.set_title("Grid-Random: %s (%s)" % (input_type, operation))

            xlims = [a*b for (a,b) in zip([.95, 1.05], [np.min(X), np.max(X)])]
            ax.set_xlim(xlims) 

            m = cm.ScalarMappable(cmap=cm.jet)
            m.set_array(2*temperatures[t0:])
            fig.colorbar(m,  ax=axs.ravel().tolist(), orientation='horizontal', label='temperature',
                        fraction=0.046) # , pad=0.04

#             handles, labels = axs[0].get_legend_handles_labels()
#             plt.figlegend( handles, labels, loc = 'upper center', ncol=5, labelspacing=0. )

            plt.show() 

#                 fig_name = '%s/%s_%s_%s.png' % (fig_out_dir, fig_out_root, measure, input_type, network)
#                 fig_files.append(fig_name)
#                 savefig(fig_name)



#     if len(fig_files):
#         call(["convert"] + fig_files + ['-append', '%s/%s.png' % (fig_out_dir, fig_out_root)])


In [ ]:
##########################################################################################################################
## Overlay all different MI measures regardless of \Phi
##########################################################################################################################

import scipy 

fig_out_root = 'corr_phi_mi_diff_no_input'

# t0 = 2
t0 = 0

mi_input_files = ['measure_d1_mice_grid_rand_non_test_T_0.0_2.8_new2.pickle']

# mi_input_files = ['measure_d1_mice_grid_non_N_9_2d_taus_0_1_5.pickle']
trange = [0., 2.5]
ntemp = 12
temperatures = np.linspace(trange[0], trange[1], ntemp)

# mi_input_files = ['measure_d1_v2_grid_rand_non_test_T_0.0_2.8.pickle',
#                  'measure_dte_v2_grid_rand_non_test_T_0.0_2.8.pickle']

# mi_input_files = ['measure_d1_mice_grid_rand_non_test_T_0.0_2.8_new.pickle',
# 'measure_d1_cause_grid_rand_non_test_T_0.0_2.8_new.pickle',
# 'measure_d1_effect_grid_rand_non_test_T_0.0_2.8_new.pickle']

# mi_input_files = ['measure_d1_v2_grid_rand_non_test_T_0.0_2.8.pickle']
# mi_input_files = ['measure_dte_v2_grid_rand_non_test_T_0.0_2.8.pickle']

# mi_input_files = ['measure_d1_v2_grid_rand_non_test_T_0.0_2.8.pickle',
#                  'measure_dte_v2_grid_rand_non_test_T_0.0_2.8.pickle']

# mi_input_files = ['measure_dte_v2_grid_rand_non_test_T_0.5_1.0.pickle']

markers = {'Grid': 'o-', 'Random': 'x--'}
fcolor = {'Grid': m, 'Random': 'none'}

use_average_mi = True
# use_average_mi = False

input_types = ['non']

def sum_mi_over_mechanisms(all_measures):
    
    all_mi = all_measures[2]

    ntemp = len(all_mi)
    nruns = len(all_mi[0])
    maxmec = len(all_mi[0][0][imn])+1

    Y_mean = [np.mean([np.sum(all_mi[t][r][imn]) for r in range(nruns)]) for t in range(ntemp)]    
    Y_std = [np.std([np.sum(all_mi[t][r][imn]) for r in range(nruns)]) for t in range(ntemp)]  

#     Y_mean = [np.mean([np.sum(m/n for m,n in zip(all_mi[t][r][imn], range(1,maxmec))) for r in range(nruns)]) for t in range(ntemp)]    
#     Y_std = [np.std([np.sum(m/n for m,n in zip(all_mi[t][r][imn], range(1,maxmec))) for r in range(nruns)]) for t in range(ntemp)]    

    return Y_mean, Y_std

def log_sum_mi_over_mechanisms(all_measures):
    Y_mean, Y_std = sum_mi_over_mechanisms(all_measures)
    return np.log(Y_mean), Y_std

def log_degeneracy(all_measures):
    return np.log(np.abs(all_measures[0])), all_measures[1]

def degeneracy(all_measures):
    return all_measures[0], all_measures[1]

# all_measures = [sum_mi_over_mechanisms, degeneracy]
all_measures = [sum_mi_over_mechanisms, log_sum_mi_over_mechanisms, degeneracy, log_degeneracy]
# all_measures = [sum_mi_over_mechanisms]

all_phi = {'non': (all_results, all_results_std)}
    
measure_markers = ['s-', 'o--', 'p:']
all_mi_classes = dict()
for (imi, mi_input_file) in enumerate(mi_input_files):

    with open('%s/%s' % (mi_input_dir, mi_input_file), 'rb') as f:
        mis = pickle.load(f)

    measure_class = '_'.join(mi_input_file.split('_')[:3])

    all_mi = {tau: {'Grid': {input_type: mis[tau][input_type + '_grid'] for input_type in input_types}}
    
#     all_mi = {tau: {'Grid': {input_type: mis[tau][input_type + '_grid'] for input_type in input_types}, 
#                     'Random': {input_type: mis[tau][input_type + '_rand'] for input_type in input_types}}
                  for tau in mis}
    
    all_mi_classes[measure_class] = all_mi

for measure in all_measures:

    # correlate \Phi, \sum \phi and their product with MI-measures with 
    for input_type in input_types:

        fig = plt.figure(figsize = (17,6))
        
        for (imi, measure_class) in enumerate(all_mi_classes):
            
            plt.axes().set_prop_cycle(None)
                
            all_mi = all_mi_classes[measure_class]
            
            fig_files = []
        
#             ax = axs[imi]
#             ax.set_xlabel(measure_class)

            for (itau, tau) in enumerate(mis):

                Y, Y_err = measure(all_mi[tau]['Grid'][input_type])
                label = "%s: tau %d" % (measure_class, tau)
                ax = plt.errorbar(temperatures[t0:], Y[t0:], yerr=Y_err[t0:], fmt=measure_markers[imi], label=label)

        handles, labels = plt.axes().get_legend_handles_labels()
        plt.legend(handles, labels)
        plt.ylabel(measure.__name__)
        plt.title("Different MIs for the Grid")

#         xlims = [a*b for (a,b) in zip([.95, 1.05], [np.min(X), np.max(X)])]
#         ax.set_xlim(xlims) 

#         m = cm.ScalarMappable(cmap=cm.jet)
#         m.set_array(2*temperatures[t0:])
#         fig.colorbar(m,  ax=axs.ravel().tolist(), orientation='horizontal', label='temperature',
#                     fraction=0.046) # , pad=0.04

        plt.show() 
